In [ ]:
# importing the necessary libraries
import gym
import myosuite
from stable_baselines3 import PPO
from gym.envs.registration import register
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from time import sleep


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# environment registration
ENV_NAME = 'Reach-v0'
MODEL_PATH = "C:/Personal/Second year/Sem 2/Project Course/Submission/arm/myoarm.xml"
register(id=ENV_NAME,
        entry_point='myosuite.envs.myo.myobase.reach_v0:ReachEnvV0',
        max_episode_steps=200,
        kwargs={
            'model_path': MODEL_PATH,
            'target_reach_range': {'ECU-P3': ((-0.04, -0.25, 0.04), (-0.04, -0.25, 0.04))},
            # 'target_reach_range': {'ECU-P3': ((0.04, -0.45, 0.03), (0.04, -0.45, 0.03))},
            # 'target_reach_range': {'ECU-P3': ((0.05, -0.45, 0.02), (0.05, -0.45, 0.02))},
            # 'target_reach_range': {'ECU-P3': ((0.04, -0.45, 0.01), (0.04, -0.45, 0.01))},
            'normalize_act': True,
        }
)
#-0.01062, -0.532, 0.05958
# -0.017, -0.45, 0.07
# -0.09, 0.15, 0.561

In [ ]:
# initialising the environment
env = gym.make('Reach-v0', seed=1)
obs = env.reset()

# initialising the evaluation environment
eval_env = gym.make('Reach-v0', seed=1)
eval_env.reset()

# Callback to test the performance
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=True, render=False)

In [ ]:
env.sim

In [ ]:
env.get_env_state()

In [ ]:
env.sim.data.site('ECU-P3')

In [ ]:
# arrays for storing the coordinates of the trajectory
observations_x=[env.sim.data.site('ECU-P3').xpos[0]]
observations_y=[env.sim.data.site('ECU-P3').xpos[1]]
observations_z=[env.sim.data.site('ECU-P3').xpos[2]]

In [ ]:
# RL model
model = PPO("MlpPolicy", env, verbose=1, device=device)
model.learn(total_timesteps=10000, callback=eval_callback)
model.save("myoarm_model")
model.load("myoarm_model")

# loop to carry out the simulation
done = False
# i = 1
while not done:
    act, _ = model.predict(obs)
    sleep(0.1)
    obs, reward, done, info = env.step(act)
    observations_x.append(env.sim.data.site('ECU-P3').xpos[0])
    observations_y.append(env.sim.data.site('ECU-P3').xpos[1])
    observations_z.append(env.sim.data.site('ECU-P3').xpos[2])
    env.mj_render()
    print(done, act, obs)
    print(env.sim.data.time)
    # i += 1

In [ ]:
#plotting the coordinates of the trajectory
plt.plot(observations_x[0], 'o')
plt.plot(observations_x)

In [ ]:
plt.plot(observations_y[0], 'o')
plt.plot(observations_y)

In [ ]:
plt.plot(observations_z[0], 'o')
plt.plot(observations_z)